# 导入库

In [189]:
import pandas as pd
import locale
from collections import defaultdict
import pycountry
import scipy.sparse as ss
import itertools
import cPickle
import hashlib
import scipy.io as sio
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 导入数据

In [11]:
print "......现在开始载入数据......"
filename = ["event_attendees.csv","events.csv","test.csv","train.csv","users.csv",
             "user_friends.csv"]

reader = pd.read_csv(filename[0],iterator=True)
try:
    event_attendees = reader.get_chunk(10000000)
    print "载入%s成功"%filename[0]
except StopIteration:
    print "Iteration is stopped"
    
reader = pd.read_csv(filename[1],iterator=True)
try:
    dfevents = reader.get_chunk(10000000)
    print "载入%s成功"%filename[1]
except StopIteration:
    print "Iteration is stopped"
    
reader = pd.read_csv(filename[2],iterator=True)
try:
    test = reader.get_chunk(10000000)
    print "载入%s成功"%filename[2]
except StopIteration:
    print "Iteration is stopped"    
    
reader = pd.read_csv(filename[3],iterator=True)
try:
    train = reader.get_chunk(10000000)
    print "载入%s成功"%filename[3]
except StopIteration:
    print "Iteration is stopped"    
    
reader = pd.read_csv(filename[4],iterator=True)
try:
    dfusers = reader.get_chunk(10000000)
    print "载入%s成功"%filename[4]
except StopIteration:
    print "Iteration is stopped"
    
reader = pd.read_csv(filename[5],iterator=True)
try:
    user_friends = reader.get_chunk(10000000)
    print "载入%s成功"%filename[5]
except StopIteration:
    print "Iteration is stopped"

......现在开始载入数据......
载入event_attendees.csv成功
载入events.csv成功
载入test.csv成功
载入train.csv成功
载入users.csv成功
载入user_friends.csv成功


# 数据清洗

### 处理users

In [281]:
print "现在开始对users的数据进行清洗和转换"

现在开始对users的数据进行清洗和转换


In [282]:
print '查看users数据'
dfusers.head()

查看users数据


,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [283]:
'''users的locale需要转为数字，利用locale这个包'''
print "利用locale包处理users.locale"
localemap = defaultdict(int)
for i, l in enumerate(locale.locale_alias.keys()):
    localemap[l] = i
dfusers.locale = dfusers.locale.apply(lambda x: localemap[x.lower()])

利用locale包处理users.locale


In [284]:
print "把users.birthyear转为int类型，然后看一下是否存在异常值和空值"
try:
    dfusers.birthyear = dfusers.birthyear.apply(lambda x: int(x))
except Exception as e:
    print e

把users.birthyear转为int类型，然后看一下是否存在异常值和空值
invalid literal for int() with base 10: 'None'


In [285]:
'''users.birthyear存在不正规的数据，用正规年份表示的转为int类型，否则全部为平均年份'''
def getbirthyear(x):
    try:
        return int(x)
    except:
        return 0
dfusers.birthyear = dfusers.birthyear.apply(lambda x: getbirthyear(x))
dfusers_birthyear_avg = float(dfusers.birthyear.sum()/np.array(dfusers.birthyear).nonzero()[0].shape[0])
def getbirthyear_avg(x):
    if x > 0:
        return x
    else:
        return users_birthyear_avg
dfusers.birthyear = dfusers.birthyear.apply(lambda x: getbirthyear_avg(x))

In [286]:
dfusers.gender.value_counts()
print "gender的数据全部正常，只需要转为0，1就可以了"
dfusers['female'] = pd.get_dummies(dfusers.gender)['female']
dfusers['male'] = pd.get_dummies(dfusers.gender)['male']

gender的数据全部正常，只需要转为0，1就可以了


In [287]:
print "处理joinedat，提取年和月"
print sum(dfusers.joinedAt.isnull())
print "存在缺失值"
def getyear(x):
    try:
        return datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ").year
    except:
        return 0
def getmonth(x):
    try:
        return datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ").month
    except:
        return 0
dfusers['year'] = dfusers.joinedAt.apply(lambda x: getyear(x))
dfusers['month'] = dfusers.joinedAt.apply(lambda x: getmonth(x))

处理joinedat，提取年和月
57
存在缺失值


In [288]:
print "处理locaiton"
countryIdMap = defaultdict(int)
ctryIdx = defaultdict(int)
for i, c in enumerate(pycountry.countries):
    countryIdMap[c.name.lower()] = i + 1
    if c.name.lower() == "usa":
        ctryIdx["US"] = i
    if c.name.lower() == "canada":
        ctryIdx["CA"] = i
for cc in ctryIdx.keys():
    for s in pycountry.subdivisions.get(country_code=cc):
        countryIdMap[s.name.lower()] = ctryIdx[cc] + 1
def getCountryId(location):
    if (isinstance(location, str)
        and len(location.strip()) > 0
        and location.rfind("  ") > -1):
          return countryIdMap[location[location.rindex("  ") + 2:].lower()]
    else:
          return 0
dfusers['location'] = dfusers.location.apply(lambda x: getCountryId(x))

处理locaiton


In [289]:
print "处理timezone"
def getTimezoneInt(timezone):
    try:
        return int(timezone)
    except:
        return 0
dfusers.timezone = dfusers.timezone.apply(lambda x: getTimezoneInt(x))

处理timezone


### 处理events

In [290]:
print '查看events数据'
dfevents.head()

查看events数据


,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


In [291]:
dfevents['year'] = dfevents.start_time.apply(lambda x: getyear(x))
dfevents['month'] = dfevents.start_time.apply(lambda x: getmonth(x))

In [292]:
dfevents = dfevents.drop(['zip'],axis=1)

In [293]:
print "处理city， state， zip， country 不知道为什么要这么干"
def getFeatureHash(value):
    try:
        return int(hashlib.sha224(value).hexdigest()[0:4], 16)
    except:
        return -1 
dfevents.city = dfevents.city.apply(lambda x: getFeatureHash(x))
dfevents.state = dfevents.state.apply(lambda x: getFeatureHash(x))
dfevents.country = dfevents.country.apply(lambda x: getFeatureHash(x))

处理city， state， zip， country 不知道为什么要这么干


In [392]:
print "处理lat, lng"
def getFloatValue(value):
    try:
        return float(value)

    except:
        return 0.01
#dfevents.lat = dfevents.lat.fillna(0)
#dfevents.lng = dfevents.lng.fillna(0)
dfevents.lat = dfevents.lat.apply(lambda x: getFloatValue(x))
dfevents.lng = dfevents.lng.apply(lambda x: getFloatValue(x))

处理lat, lng


## 建模，构建相似矩阵

### 统计训练集和测试集中有多少独立的用户

In [250]:
print '''
得到一个user_eventscores矩阵，行是user，列是event，数字是user对event的打分

只有一个user参加过的event以及只参加过一个event的user不考虑
'''


得到一个user_eventscores矩阵，行是user，列是event，数字是user对event的打分

只有一个user参加过的event以及只参加过一个event的user不考虑



In [229]:
train.head()

,user,event,invited,timestamp,interested,not_interested,testtrain
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,train
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,train
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0,train
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0,train
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0,train


In [230]:
uniqueusers = train.user.unique()
uniqueevent = train.event.unique()

In [260]:
eventforuser = defaultdict(set)
userforevent = defaultdict(set)
for i in range(len(train)):
    user = train.user[i]
    event = train.event[i]
    eventforuser[user].add(event)
    userforevent[event].add(user)
userindex = dict()
eventindex = dict()
for index, user in enumerate(uniqueusers):
    userindex[user] = index
for index, event in enumerate(uniqueevent):
    eventindex[event] = index

In [271]:
usereventscores = np.zeros((len(uniqueusers),len(uniqueevent)))
for i in range(len(train)):
    user = train.user[i]
    event = train.event[i]
    row = userindex[user]
    col = eventindex[event]
    usereventscores[row, col] = train.interested[i] -train.not_interested[i]
print "usereventscores矩阵完成"

usereventscores矩阵完成


In [277]:
uniqueuserpair = set()
uniqueeventpair = set()
for user in eventforuser:
    events = eventforuser[user]
    if len(events) > 2:
        uniqueeventpair.update(itertools.combinations(events, 2))
for event in userforevent:
    users = userforevent[event]
    if len(users) > 2:
        uniqueuserpair.update(itertools.combinations(users, 2))
print "已把只有一人参与的event和只参与一个活动的user除去"

已把只有一人参与的event和只参与一个活动的user除去


### 构建user-user相似度矩阵

In [295]:
from sklearn.preprocessing import normalize

In [298]:
dfusers = dfusers.drop(['gender','joinedAt'],axis=1)

In [330]:
print '''
不明白为什么离散的locale，country都可以正则化和怎么求余弦相似度
'''


不明白为什么离散的locale，country都可以正则化和怎么求余弦相似度



In [329]:
dfusers.head()

,user_id,locale,birthyear,location,timezone,female,male,year,month
0,3197468391,229,1993.0,104,480,0.0,1.0,2012,10
1,3537982273,229,1992.0,104,420,0.0,1.0,2012,9
2,823183725,319,1975.0,40,-240,0.0,1.0,2012,10
3,1872223848,319,1991.0,0,210,1.0,0.0,2012,11
4,3429017717,229,1995.0,0,420,1.0,0.0,2012,9


In [439]:
#只考虑train.csv中出现过的user
usermatrix = np.zeros((len(uniqueusers),len(dfusers.columns)-1))
for i in range(len(dfusers)):
    user = dfusers.user_id[i]
    if userindex.has_key(user):
        row = userindex[user]
        usermatrix[row,0] = dfusers.locale[row]
        usermatrix[row,1] = dfusers.birthyear[row]
        usermatrix[row,2] = dfusers.location[row]
        usermatrix[row,3] = dfusers.timezone[row]
        usermatrix[row,4] = dfusers.female[row]
        usermatrix[row,5] = dfusers.male[row]
        usermatrix[row,6] = dfusers.year[row]
        usermatrix[row,7] = dfusers.month[row]

In [440]:
#归一化矩阵
usermatrix = normalize(usermatrix, norm="l1", axis=0, copy=False)

In [441]:
#皮尔森相似度求user-user相似度矩阵,为了方便和速度，只计算uniqueuserpair中的
import scipy.spatial.distance as ssd
usersimmatrix = np.zeros((len(userindex),len(userindex)))
for i in range(0,usersimmatrix.shape[0]):
    usersimmatrix[i, i] = 1.0
for user1, user2 in uniqueuserpair:
    row = userindex[user1]
    col = userindex[user2]
    sim = ssd.correlation(usermatrix[row],usermatrix[col])
    usersimmatrix[row, col] = sim
    usersimmatrix[col, row] = sim

### 构建event-event相似矩阵

In [374]:
print """
      构建event-event相似度，注意这里有2种相似度：
      1）由用户-event行为，类似协同过滤算出的相似度
      2）由event本身的内容(event信息)计算出的event-event相似度
    """


      构建event-event相似度，注意这里有2种相似度：
      1）由用户-event行为，类似协同过滤算出的相似度
      2）由event本身的内容(event信息)计算出的event-event相似度
    


In [638]:
dfevents.tail()

,event_id,user_id,start_time,city,state,country,lat,lng,c_1,c_2,...,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other,year,month
3137967,3206911916,3817662916,2012-06-07T14:00:00.000Z,-1,-1,-1,45.238,11.754,0,0,...,0,0,0,0,0,0,0,45,2012,6
3137968,3298484812,2119960237,2014-02-26T02:00:00.002Z,-1,-1,-1,0.000,0.000,0,0,...,0,0,0,0,0,0,0,8,2014,2
3137969,1331217755,1149466221,2012-12-02T07:30:00.002Z,-1,-1,-1,0.000,0.000,0,0,...,0,0,0,0,0,0,0,2,2012,12
3137970,1627348069,3748671867,2012-10-20T04:00:00.003Z,44247,51357,20893,33.857,-118.391,2,3,...,1,0,0,0,0,1,1,51,2012,10
3137971,1373240562,3334738518,2013-02-14T20:00:00.002Z,-1,-1,-1,0.000,0.000,0,0,...,0,0,0,0,0,0,0,1,2013,2


In [645]:
#只考虑train.csv中出现过的event
eventpropmatrix = np.zeros((len(dfevents),7))
for i in range(len(dfusers)):
    event = dfevents.event_id[i]
    if eventindex.has_key(event):
        row = eventindex[event]
        eventpropmatrix[row,0] = dfevents.city[i]
        eventpropmatrix[row,1] = dfevents.state[i]
        eventpropmatrix[row,2] = dfevents.country[i]
        eventpropmatrix[row,3] = dfevents.lat[i]
        eventpropmatrix[row,4] = dfevents.lng[i]
        eventpropmatrix[row,5] = dfevents.year[i]
        eventpropmatrix[row,6] = dfevents.month[i]
#归一化矩阵
eventpropmatrix = normalize(eventpropmatrix, norm="l1", axis=0, copy=False)

In [630]:
eventcontmatrix = np.zeros((len(dfevents),101))
for i in range(len(dfevents)):
    event = dfevents.event_id[i]
    if eventindex.has_key(event):
        row = eventindex[event]
        for j in range(0,101):
            colname = dfevents.columns[j+8]
            eventcontmatrix[row, j] = dfevents[colname][i]
#归一化矩阵
eventcontmatrix = normalize(eventcontmatrix, norm="l1", axis=0, copy=False)

In [ ]:
'''
  皮尔森相似度和修正的余弦相似度的区别在于分母，皮尔森相似度长度是i，j同时打分的项目的长度，余弦是i打分的项目和j打分的项目）
  1.如果数据密集（所有数据几乎都有属性值，属性值量级重要），就用欧几里德算法
  2.数据受级别膨胀影响（不同的用户使用不同的评分标准），就用皮尔逊相关系数或者修正的余弦相似度算法
  3.数据稀疏性强，就考虑用夹角余弦相似度算法
  
  因为这里是计算event-event的相似度，event的属性是客观的，不受用户评分习惯的影响，因此可以用余弦相似度
'''

In [651]:
#计算event-event相似度
eventpropsimmatrix = np.zeros((len(uniqueevent),len(uniqueevent)))
for i in range(0,eventpropsimmatrix.shape[0]):
    eventpropsimmatrix[i, i] = 1.0
for event1, event2 in uniqueeventpair:
    row = eventindex[event1]
    col = eventindex[event2]
    sim = ssd.cosine(eventpropmatrix[row],eventpropmatrix[col])
    eventpropsimmatrix[row, col] = sim
    eventpropsimmatrix[col, row] = sim
eventcontsimmatrix = np.zeros((len(uniqueevent),len(uniqueevent)))
for i in range(0,eventcontsimmatrix.shape[0]):
    eventcontsimmatrix[i, i] = 1.0
for event1, event2 in uniqueeventpair:
    row = eventindex[event1]
    col = eventindex[event2]
    sim = ssd.cosine(eventcontmatrix[row],eventcontmatrix[col])
    eventcontsimmatrix[row, col] = sim
    eventcontsimmatrix[col, row] = sim

In [649]:
eventpropsimmatrix

array([[  1.,  nan,  nan, ...,   0.,   0.,   0.],
       [ nan,   1.,  nan, ...,   0.,   0.,   0.],
       [ nan,  nan,   1., ...,   0.,   0.,   0.],
       ..., 
       [  0.,   0.,   0., ...,   1.,  nan,  nan],
       [  0.,   0.,   0., ...,  nan,   1.,  nan],
       [  0.,   0.,   0., ...,  nan,  nan,   1.]])

In [447]:
print '查看user_friends数据'
user_friends.head()

查看user_friends数据


,user,friends
0,3197468391,1346449342 3873244116 4226080662 1222907620 54...
1,3537982273,1491560444 395798035 2036380346 899375619 3534...
2,823183725,1484954627 1950387873 1652977611 4185960823 42...
3,1872223848,83361640 723814682 557944478 1724049724 253059...
4,3429017717,4253303705 2130310957 1838389374 3928735761 71...


In [448]:
print "处理user_friends数据"

处理user_friends数据


In [513]:
"user-friend矩阵表示user朋友多少和朋友是否乐意参与活动，row是user，col是user的朋友，第i行第j列是user i的j朋友对他的影响"
userfriendmatrix = np.zeros((len(userindex),len(userindex)))
for i in range(len(user_friends)):
    user = user_friends.user[i]
    if userindex.has_key(user):
        row = userindex[user]
        friendlist = user_friends.friends[i].split(" ")
        for friend in friendlist:
            if userindex.has_key(friend):
                j = userindex[friend]
                sim = float(usereventscores[i].sum())/usereventscores[i].shape[0]                           
                userfriendmatrix[row,j] = sim

In [479]:
'''
userfriendmatrix 全部为0,也就是说user的朋友没有在训练集的，只能通过统计user有多少个朋友来判断是否外向，
但这样是不怎么合理的，因为一个user可以有很多不爱参加活动的朋友
'''

In [548]:
userfriendcount = np.zeros(len(userindex))
for i in range(len(user_friends)):
    user = user_friends.user[i]
    if userindex.has_key(user):
        row = userindex[user]
        friendlist = user_friends.friends[i].split(" ")
        userfriendcount[row] = len(friendlist)
#归一化矩阵
userfriendcount = normalize(userfriendcount,norm='l1')

/Users/houxiaohui/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [543]:
print '查看event_attendees数据'
event_attendees = event_attendees.fillna(0)
event_attendees.head()

查看event_attendees数据


,event,yes,maybe,invited,no
0,1159822043,1975964455 252302513 4226086795 3805886383 142...,2733420590 517546982 1350834692 532087573 5831...,1723091036 3795873583 4109144917 3560622906 31...,3575574655 1077296663
1,686467261,2394228942 2686116898 1056558062 3792942231 41...,1498184352 645689144 3770076778 331335845 4239...,1788073374 733302094 1830571649 676508092 7081...,0
2,1186208412,0,3320380166 3810793697,1379121209 440668682,1728988561 2950720854
3,2621578336,0,0,0,0
4,855842686,2406118796 3550897984 294255260 1125817077 109...,2671721559 1761448345 2356975806 2666669465 10...,1518670705 880919237 2326414227 2673818347 332...,3500235232


In [554]:
print "处理event_attendees数据,计算event流行度矩阵"
eventpopularity = np.zeros((len(eventindex),1))
for i in range(len(event_attendees)):
    event = event_attendees.event[i]
    if eventindex.has_key(event):
        row = eventindex[event]
        try:
            num_yes = len(event_attendees.yes[i].split(" "))
        except:
            num_yes = 0
        try:
            num_no = len(event_attendees.no[i].split(" "))
        except:
            num_no = 0
        eventpopularity[row,0] = num_yes-num_no
eventpopularity = normalize( eventpopularity,norm='l2')

处理event_attendees数据,计算event流行度矩阵


## 利用协同过滤进行推荐

In [632]:
print "不懂为什么要减去原来的usereventscores"
#userbaseCF = usereventscores.T.dot(usersimmatrix).T - usereventscores
itembaseCFprob = usereventscores.dot(eventpropsimmatrix)- usereventscores
itembaseCFcont = usereventscores.dot(eventcontsimmatrix)- usereventscores

不懂为什么要减去原来的usereventscores


In [711]:
print '查看train数据'
train.head()

查看train数据


,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0


In [713]:
'''
itemprobcf太稀疏
'''
def getdata(table):
    usercflist = []
    itemprobcflist = []
    itemcontcflist = []
    eventpopularitylist = []
    userfriendlist = []
    for i in range(len(table)):
        user = table.user[i]
        event = table.event[i]
        uix = userindex[user]
        eix = eventindex[event]
        usercflist.append(userbaseCF[uix, eix])
        itemprobcflist.append(itembaseCFprob[uix, eix])
        itemcontcflist.append(itembaseCFcont[uix, eix])
        eventpopularitylist.append(eventpopularity[eix][0])
        userfriendlist.append(userfriendcount[0][uix])
    table['usercf'] = usercflist
    table['itemprobcf'] = itemprobcflist
    table['itemcontcf'] = itemcontcflist
    table['eventpopularity'] = eventpopularitylist
    table['userfriend'] = userfriendlist

In [714]:
getdata(train)

In [715]:
print sum(train.itemprobcf.notnull())
print  "item prob 协同过滤计算的相似度过于稀疏，只有8个值，因此drop这一列"
train = train.drop(['itemprobcf'],axis=1)

8
item prob 协同过滤计算的相似度过于稀疏，只有8个值，因此drop这一列


## 建模与预测 

In [677]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [663]:
train.head()

,user,event,invited,timestamp,interested,not_interested,testtrain,usercf,itemcontcf,eventpopularity,userfriend
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,train,0.000000,0.973027,-1.0,0.000389
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,train,0.000000,0.551433,1.0,0.000389
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0,train,95.937195,0.000000,1.0,0.000389
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0,train,0.000000,0.802974,1.0,0.000389
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0,train,19.063354,0.979817,1.0,0.000389


In [716]:
features = ['invited','usercf','itemcontcf','eventpopularity','userfriend']
y = train
x = train[features]
clf = SGDClassifier(loss="log", penalty="l2")

In [710]:
kf = KFold(len(train),5)
accuracy  = 0.0
X = np.matrix(x)
for u, v in kf:
    X_train, X_test, y_train, y_test = X[u], X[v], y[u], y[v]
    clf.fit(X_train,y_train)
    y_hat = clf.predict(X_test)
    accuracy += accuracy_score(y_test,y_hat)
print "正确率为{}%".format(accuracy/5*100)

正确率为63.0584415584%


可以改进的地方：
1. 通过location获得经纬度，从而算出event和user的距离,通过反馈发现，location distance很重要
2. event_popularity通过参加的人数-不参加的人数算出，但interested和invited也应该考虑
   ”yes“的人数+0.5*”maybe“的人数+0.2*”invited“的人数-”no“的人数
3. 利用user_friends 和 event_attendees计算相似度 
    event1和event2相似因为yes，maybe，invited的user有很大的交集
    user1和user2相似是因为user的yes，maybe，invited的event有很大的交集
4. timestamp可以抽取是否周末，活动邀请时间和user看到的时间，是否节假日，是否夜晚
5. l1正则化改为L2正则化
6. events中的c1-cother可以先聚类或者先pca